## 1. Import Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import pickle
import os

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

print("\nLibraries imported successfully!")

## 2. Load Dataset

In [ ]:
# Load dataset
print("Loading dataset...")
df = pd.read_csv('../../data/cleaned_label.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nSentiment distribution:")
print(df['sentiment_label'].value_counts())
print(f"\nFirst few rows:")
print(df.head())

## 3. Initialize HateBERT Model

In [ ]:
# Model name - HateBERT
MODEL_NAME = 'GroNLP/hateBERT'

print(f"Loading HateBERT model: {MODEL_NAME}")
print("Note: HateBERT is BERT re-trained on offensive/abusive content")
print("      - Based on BERT-base architecture")
print("      - Pre-trained on Reddit banned communities (1.5M posts)")
print("      - Specialized for hate speech, offensive language, abuse")
print("      - Better at understanding toxic/negative sentiment")
print("      - May capture subtle negative nuances better\n")

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)

# Move to device
model = model.to(device)
model.eval()  # Set to evaluation mode

print(f"Model loaded successfully!")
print(f"Vocabulary size: {tokenizer.vocab_size}")
print(f"Hidden size: {model.config.hidden_size}")
print(f"Number of layers: {model.config.num_hidden_layers}")
print(f"Number of attention heads: {model.config.num_attention_heads}")
print(f"Total parameters: ~110M (same as BERT-base)")

## 4. Extract Embeddings Function

In [ ]:
def get_hatebert_embedding(text, tokenizer, model, device, max_length=128):
    """
    Extract HateBERT [CLS] token embedding
    
    Args:
        text: str - Input text
        tokenizer: HateBERT tokenizer
        model: HateBERT model
        device: torch device
        max_length: int - Max sequence length
        
    Returns:
        numpy array: 768-dimensional embedding vector
    """
    # Tokenize
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Get embeddings
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # Extract [CLS] token embedding (first token)
        cls_embedding = outputs.last_hidden_state[:, 0, :]
    
    return cls_embedding.cpu().numpy().flatten()

print("Embedding extraction function defined!")

# Test on sample texts (including negative sentiment)
sample_texts = [
    "This movie is absolutely fantastic!",
    "Terrible waste of time, complete garbage.",
    "I absolutely hated this movie, worst ever."
]

print("\nTesting with sample texts:")
for text in sample_texts:
    embedding = get_hatebert_embedding(text, tokenizer, model, device)
    print(f"  '{text}' -> shape: {embedding.shape}")

## 5. Extract Embeddings for All Documents

In [ ]:
# Parameters
MAX_LENGTH = 128
BATCH_SIZE = 32  # Process in batches to save memory

print("Extracting HateBERT embeddings for all documents...")
print(f"Model: {MODEL_NAME}")
print(f"Max length: {MAX_LENGTH}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Total documents: {len(df)}\n")
print("Note: HateBERT may capture negative sentiment nuances better")
print("      due to training on offensive/toxic content\n")

embeddings = []

# Process in batches
for i in tqdm(range(0, len(df), BATCH_SIZE)):
    batch_texts = df['review_text'].iloc[i:i+BATCH_SIZE].values
    
    for text in batch_texts:
        embedding = get_hatebert_embedding(str(text), tokenizer, model, device, MAX_LENGTH)
        embeddings.append(embedding)

# Convert to numpy array
embeddings = np.array(embeddings)

print(f"\nEmbeddings extracted!")
print(f"Shape: {embeddings.shape}")
print(f"Each document: {embeddings.shape[1]}-dimensional vector")

## 6. Save Embeddings

In [ ]:
# Create output directory
output_dir = '../../models/hatebert_embeddings'
os.makedirs(output_dir, exist_ok=True)

# Save embeddings
embeddings_path = os.path.join(output_dir, 'hatebert_embeddings.npy')
np.save(embeddings_path, embeddings)
print(f"Embeddings saved: {embeddings_path}")

# Save labels
labels_path = os.path.join(output_dir, 'labels.npy')
np.save(labels_path, df['sentiment_label'].values)
print(f"Labels saved: {labels_path}")

# Save metadata
metadata = {
    'model_name': MODEL_NAME,
    'model_type': 'hateBERT (BERT-base retrained)',
    'hidden_size': model.config.hidden_size,
    'max_length': MAX_LENGTH,
    'num_documents': len(embeddings),
    'embedding_shape': embeddings.shape,
    'extraction_method': '[CLS] token from last_hidden_state',
    'training_data': 'Reddit banned communities (1.5M posts)',
    'specialization': 'Hate speech, offensive language, toxic content',
    'notes': 'May perform better on negative/toxic sentiment due to specialized training'
}

metadata_path = os.path.join(output_dir, 'metadata.pkl')
with open(metadata_path, 'wb') as f:
    pickle.dump(metadata, f)
print(f"Metadata saved: {metadata_path}")

print("\n" + "="*80)
print("All files saved successfully!")
print("="*80)
print(f"\nOutput directory: {output_dir}")
print(f"Files:")
print(f"  - hatebert_embeddings.npy    (Embeddings: {embeddings.shape})")
print(f"  - labels.npy                 (Sentiment labels)")
print(f"  - metadata.pkl               (Model metadata)")

## 7. Verify Saved Embeddings

In [ ]:
# Load saved embeddings
print("Loading saved embeddings...\n")

loaded_embeddings = np.load(embeddings_path)
loaded_labels = np.load(labels_path)

with open(metadata_path, 'rb') as f:
    loaded_metadata = pickle.load(f)

print(f"✓ Embeddings loaded: {loaded_embeddings.shape}")
print(f"✓ Labels loaded: {loaded_labels.shape}")
print(f"\n✓ Metadata:")
for key, value in loaded_metadata.items():
    print(f"    {key}: {value}")

# Verify integrity
print(f"\n✓ Verification:")
print(f"    Embeddings match: {np.allclose(embeddings, loaded_embeddings)}")
print(f"    Labels match: {np.array_equal(df['sentiment_label'].values, loaded_labels)}")

## 8. Embedding Statistics

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print("Embedding Statistics:\n")

# Basic stats
print(f"Shape: {embeddings.shape}")
print(f"Mean: {embeddings.mean():.4f}")
print(f"Std: {embeddings.std():.4f}")
print(f"Min: {embeddings.min():.4f}")
print(f"Max: {embeddings.max():.4f}")

# Plot distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Overall distribution
axes[0].hist(embeddings.flatten(), bins=100, alpha=0.7, color='darkred')
axes[0].set_xlabel('Embedding Value')
axes[0].set_ylabel('Frequency')
axes[0].set_title('HateBERT Embedding Value Distribution')
axes[0].grid(True, alpha=0.3)

# Mean embedding per document
mean_embeddings = embeddings.mean(axis=1)
axes[1].hist(mean_embeddings, bins=50, alpha=0.7, color='maroon')
axes[1].set_xlabel('Mean Embedding Value')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Mean Embedding per Document')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nStatistics plotted!")

## 9. Compare Positive vs Negative Embeddings

In [ ]:
from sklearn.decomposition import PCA

# Sample for visualization
sample_size = 1000
indices = np.random.choice(len(embeddings), size=min(sample_size, len(embeddings)), replace=False)

sample_embeddings = embeddings[indices]
sample_labels = df['sentiment_label'].iloc[indices].values

# PCA reduction to 2D
print(f"Performing PCA on {len(sample_embeddings)} samples...")
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(sample_embeddings)

print(f"Explained variance: {pca.explained_variance_ratio_.sum():.2%}")

# Plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    embeddings_2d[:, 0],
    embeddings_2d[:, 1],
    c=sample_labels,
    cmap='RdYlGn',
    alpha=0.6,
    s=30
)
plt.colorbar(scatter, label='Sentiment (0=Neg, 1=Pos)')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
plt.title('HateBERT Embeddings Visualization (PCA)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nVisualization complete!")
print("Note: HateBERT may show better separation for negative sentiment")

## 10. Negative Sentiment Analysis

In [ ]:
# Analyze negative sentiment embeddings
print("\n" + "="*80)
print("Negative Sentiment Analysis")
print("="*80)

neg_indices = df[df['sentiment_label'] == 0].index
pos_indices = df[df['sentiment_label'] == 1].index

neg_embeddings = embeddings[neg_indices]
pos_embeddings = embeddings[pos_indices]

print(f"\nNegative reviews: {len(neg_embeddings):,}")
print(f"Positive reviews: {len(pos_embeddings):,}")

print(f"\nNegative embedding stats:")
print(f"  Mean: {neg_embeddings.mean():.4f}")
print(f"  Std: {neg_embeddings.std():.4f}")

print(f"\nPositive embedding stats:")
print(f"  Mean: {pos_embeddings.mean():.4f}")
print(f"  Std: {pos_embeddings.std():.4f}")

# Compare distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(neg_embeddings.mean(axis=1), bins=50, alpha=0.6, label='Negative', color='red')
plt.hist(pos_embeddings.mean(axis=1), bins=50, alpha=0.6, label='Positive', color='green')
plt.xlabel('Mean Embedding Value')
plt.ylabel('Frequency')
plt.title('HateBERT: Mean Embedding Distribution by Sentiment')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot([neg_embeddings.mean(axis=1), pos_embeddings.mean(axis=1)],
            labels=['Negative', 'Positive'])
plt.ylabel('Mean Embedding Value')
plt.title('HateBERT: Embedding Comparison by Sentiment')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nHateBERT's training on toxic content may provide better negative sentiment representation")

## 11. HateBERT vs BERT Comparison

In [ ]:
print("\n" + "="*80)
print("HateBERT vs BERT Comparison")
print("="*80)

comparison = {
    'Feature': [
        'Base Architecture',
        'Parameters',
        'Hidden Size',
        'Training Data',
        'Training Domain',
        'Specialization',
        'Best For',
        'Vocabulary',
        'Performance'
    ],
    'BERT': [
        'BERT-base',
        '110M',
        '768',
        'BookCorpus + Wikipedia',
        'General text (clean)',
        'General NLP',
        'General tasks',
        'Standard English',
        'Good overall'
    ],
    'HateBERT': [
        'BERT-base (re-trained)',
        '110M',
        '768',
        'Reddit banned communities',
        'Offensive/toxic content',
        'Hate speech, abuse',
        'Toxic/negative content',
        'Includes slang, offensive terms',
        'Better on toxic/negative text'
    ]
}

comparison_df = pd.DataFrame(comparison)
print(comparison_df.to_string(index=False))

print("\n" + "="*80)
print("Why HateBERT for Sentiment Analysis:")
print("="*80)
print("\n1. Negative Sentiment Understanding:")
print("   - Trained on offensive/toxic content from Reddit")
print("   - Better at capturing subtle negative nuances")
print("   - Understands slang, informal negative expressions")

print("\n2. Movie Reviews Context:")
print("   - Reviews can be harsh/critical (similar to toxic content)")
print("   - Informal language, strong opinions")
print("   - May capture extreme negative sentiment better")

print("\n3. Vocabulary:")
print("   - Includes offensive/slang terms not in standard BERT")
print("   - Better coverage of negative expressions")
print("   - More robust to informal/harsh language")

print("\n" + "="*80)
print("Note: Despite the name, HateBERT can be used for any sentiment analysis!")
print("The toxic training data gives it better negative sentiment understanding.")
print("="*80)

## 12. Summary

In [ ]:
print("\n" + "="*80)
print("HateBERT EMBEDDING EXTRACTION SUMMARY")
print("="*80)

print(f"\n📊 Model: {MODEL_NAME}")
print(f"🚀 Architecture: BERT-base re-trained on toxic content")
print(f"📏 Embedding dimension: {embeddings.shape[1]}")
print(f"📁 Total documents: {embeddings.shape[0]:,}")
print(f"💾 Total size: {embeddings.nbytes / (1024**2):.2f} MB")
print(f"⚡ Parameters: 110M (same as BERT-base)")

print(f"\n✅ Files saved:")
print(f"   {output_dir}/")
print(f"   ├── hatebert_embeddings.npy  ({embeddings.shape})")
print(f"   ├── labels.npy               ({loaded_labels.shape})")
print(f"   └── metadata.pkl")

print(f"\n💡 Usage:")
print(f"   These embeddings can be used with any classifier:")
print(f"   - Logistic Regression")
print(f"   - SVM")
print(f"   - Random Forest")
print(f"   - Neural Networks")

print(f"\n🔧 Load embeddings:")
print(f"   X = np.load('{embeddings_path}')")
print(f"   y = np.load('{labels_path}')")

print(f"\n⚡ Why HateBERT:")
print(f"   - Trained on 1.5M Reddit posts from banned communities")
print(f"   - Specialized for offensive/toxic content")
print(f"   - Better understanding of negative sentiment nuances")
print(f"   - Includes slang and informal vocabulary")
print(f"   - May outperform standard BERT on harsh/critical reviews")
print(f"   - Good for capturing extreme negative sentiment")

print(f"\n🎯 Expected Advantage:")
print(f"   - Better at detecting subtle negative sentiment")
print(f"   - More robust to informal/harsh language")
print(f"   - May improve negative review classification")

print("\n" + "="*80)
print("✅ HateBERT embedding extraction complete!")
print("="*80)